# Checking the uroboros property, taking ambiguity into account

In [27]:
from collections import Counter

import numpy as np
import pandas as pd
import re
import urllib  # the lib that handles the url stuff

In [28]:
names=['en', 'hu', 'la', 'pl', 'id', 'uro', 'pos', 'defi', 'comment']

## Thematic-role version

In [29]:
fl = pd.read_csv('4lang', sep='\t', names=names).fillna('')

In [30]:
fl = fl[fl.defi != '']

In [31]:
concept_chars = '=@a-zA-Z_/0-9-'

In [32]:
parens_and_stuff = "[]<>,()'"

In [33]:
def find_concepts(defi):
    return re.findall(f'[{concept_chars}]+', defi)

In [34]:
defining_tokens = []
for concepts in fl.defi.astype(str).map(find_concepts).values:
    defining_tokens += concepts

In [35]:
defining_vocab = pd.Series(Counter(defining_tokens)).sort_values(ascending=False)

In [36]:
defining_vocab

HAS         688
=AGT        450
=PAT        390
lack        357
IN/2758     330
           ... 
candle        1
wait          1
trade         1
industry      1
title         1
Length: 1714, dtype: int64

In [37]:
headwords = pd.Series(Counter(fl.en.values)).sort_values(ascending=False)

In [38]:
headwords[headwords > 1].head()

hail        4
interest    3
total       3
light       3
trunk       3
dtype: int64

### Undefined/primitive words used in definitions

In [39]:
undefined = set(defining_vocab.index.str.lower()) - set(headwords.index.str.lower())

In [40]:
def is_special(defing_token):
    return re.match('^[@=]', defing_token) or '/' in defing_token

In [41]:
%pprint
print([word for word in undefined if not is_special(word)])
%pprint

Pretty printing has been turned OFF
['characteristic', 'want', 'be', 'palm', 'other', 'for', 'is_a', 'passionate', 'has', 'er', 'inherent', 'next_to', 'tired', 'part_of', 'before', 'enthusiastic', 'lack', 'not', 'target', 'celebrate', 'all', 'also', 'identity']
Pretty printing has been turned ON


### Ambig words used without disambiguation

In [42]:
headwords[headwords>1]

hail        4
interest    3
total       3
light       3
trunk       3
           ..
fat         2
like        2
deal        2
pet         2
lime        2
Length: 133, dtype: int64

In [45]:
set(defining_vocab.index).intersection(set(headwords[headwords>1].index))

set()

### Is there an erronous disambig?

In [23]:
possible_disambed_words = set(fl.apply(lambda ser: f'{ser.en}/{ser.id}', axis=1).values)
set(defining_vocab.index.str.lower()[defining_vocab.index.str.contains('/')]) - possible_disambed_words

{'case/2423',
 'close/1413',
 'develop/758',
 'for/2782',
 'in/10',
 'level/2784',
 'manner/1715',
 'march/1563',
 'may/1560',
 'miss/1357',
 'mrs/35'}

No, just casing differences.

### Words with more headwords but only one used

In [46]:
n_senses_used = pd.Series(Counter([pair[0] for pair in defining_vocab.index.str.split('/') if len(pair)>1]))
n_senses_used[n_senses_used>1].sort_values(ascending=False).head()

right    3
place    2
ring     2
stage    2
court    2
dtype: int64

In [47]:
%pprint
print(sorted(set(headwords[headwords>1].index).intersection(set(n_senses_used[n_senses_used==1].index))))
%pprint

Pretty printing has been turned OFF
['-th', 'balance', 'base', 'bathe', 'beam', 'bore', 'bow', 'cage', 'can', 'catch', 'chance', 'charge', 'club', 'comfort', 'company', 'cook', 'course', 'cure', 'desert', 'draw', 'effect', 'fast', 'figure', 'firm', 'hail', 'hair', 'horn', 'host', 'interest', 'lead', 'match', 'mean', 'minister', 'miss', 'mock', 'open', 'pet', 'post', 'regard', 'return', 'season', 'set', 'shut', 'sink', 'spirit', 'spoil', 'spring', 'steep', 'straight', 'stroke', 'succeed', 'take', 'tire', 'trick', 'weary', 'with', 'wound']
Pretty printing has been turned ON


## Current version of `kornai/4lang/4lang`

In [49]:
refrom = pd.read_csv('../4lang/4lang', sep='\t', names=names, dtype=str).fillna('')

In [50]:
refrom.loc[refrom.id=='#','id']=np.nan

In [54]:
#refrom = refrom[refrom.defi!='TODO']

In [55]:
def is_printname(token):
    return re.fullmatch('^[=a-zA-Z_0-9.#/-]+', str(token))

In [56]:
refrom.groupby('uro').size()

uro
c     110
e     358
p      14
u    2623
dtype: int64

In [57]:
refrom.groupby('pos').size()

pos
#       6
A     623
D      91
G     142
N    1611
U     116
V     516
dtype: int64

In [58]:
refrom.groupby('defi').size().sort_values(ascending=False).head()

defi
=REL        10
lack         7
good         6
thing        5
together     5
dtype: int64

In [59]:
refrom.groupby('comment').size().sort_values(ascending=False).head()

comment
%      2620
%1       72
%ND      33
%2       17
%RA      16
dtype: int64

In [60]:
refrom[~refrom.comment.str.startswith('%')]

,en,hu,la,pl,id,uro,pos,defi,comment


## Refrom/600

In [29]:
reform600 = pd.read_csv('../4lang/Reform/600', sep='\t', names=names)

In [30]:
reform600.head()

,en,hu,la,pl,id,uro,pos,defi,comment
0,-able,-hato1,-bilis,-alny,21,e,G,"gen allow {gen stem_ =agt}, ""_-able"" mark_ stem",%detestable
1,-er,-bb,-ior/-ius,-szy,14,e,G,"er_, =agt has quality, ""_-er"" mark_ stem_[qual...",%
2,-er,-o1,-tor/-trix,-ac1/ic1,3627,e,G,"stem_-er is_a =agt, ""_ -er"" mark_ stem_",%nomen agentis
3,-est,leg-bb,-issimus,naj-szyo,1513,e,G,er_ other %,NaN
4,-ing,-a1s,-endi,-anie,2,e,G,"stem_-ing is_a event, ""_-ing"" mark_ stem_",%


In [31]:
printname_pattern = '^[=a-zA-Z_0-9.#/-]+'

In [32]:
printname_pattern = '^[a-z-]+$'

In [33]:
def is_printname(token):
    return re.fullmatch(printname_pattern, str(token))

In [34]:
reform600.groupby('uro').size()

uro
c    110
e    358
p     14
u    293
dtype: int64

In [35]:
reform600.groupby('pos').size()

pos
A    144
D      6
G     31
N    440
U     24
V    130
dtype: int64

In [36]:
reform600.groupby('defi').size().sort_values(ascending=False).head()

defi
man/659                        2
lack move                      2
quantity                       2
=pat in mind, =agt has mind    2
gen want                       2
dtype: int64

In [37]:
reform600.groupby('comment').size().sort_values(ascending=False).head()

comment
%        702
%ND        9
%RA        3
%RG        2
%   %      2
dtype: int64

## V2/700

In [38]:
v2_700 = pd.read_csv('../4lang/V2/700.tsv', sep='\t')
# v2_700['def'] corresponds to fl['defi']
# v2_700.num corresponds to fl['id']

In [39]:
v2_700.head()

,en,hu,la,pl,ja,zh,num,s,l,def,%comment
0,/-able,/-ható,/-bilis,/-alni,_ -e-_,ke3- 可,21,e,G,"gen allow {gen stem_ =agt}, ""_-able"" mark_ stem",%detestable
1,/-er,/-bb,/-ior//-ius,/-szy,_ yori _ _-i,bi3jiao4 比较,14,e,G,"er_, =agt has quality, ""_-er"" mark_ stem_[qual...",%
2,/-er,/-ó,/-tor//-trix,/-arz,/-sha 者,/// -zhe3 者,3627,e,G,"stem_-er is_a =agt, ""_ -er"" mark_ stem_",%nomen agentis
3,/-est,leg-bb,/-issimus,naj...szy,ichiban 一番,zui4- 最,3625,e,G,er_ all %,NaN
4,/-est,leg...bb,/-issimus,naj...szy,ichiban _ 一番,zui4- 最,1513,e,G,er_ other %,NaN


In [40]:
printname_pattern = '^[=\w_/.#-]+$'

There is only one capitalized word, _Asia_

In [41]:
def is_printname(token):
    return re.fullmatch(printname_pattern, str(token))

In [42]:
sorted(v2_700.num.astype(int))[:20]

[2, 4, 5, 10, 12, 14, 17, 21, 29, 33, 66, 68, 72, 73, 74, 76, 78, 82, 86, 88]

In [43]:
v2_700.groupby('s').size()

s
c    110
e    359
p     14
u    293
dtype: int64

In [44]:
v2_700.groupby('l').size()

l
A    144
D      6
G     32
N    440
U     24
V    130
dtype: int64

In [45]:
v2_700.groupby('def').size().sort_values(ascending=False).head()

def
many         2
lack move    2
big          2
sex          2
gen want     2
dtype: int64

In [46]:
v2_700.groupby('%comment').size().sort_values(ascending=False).head()

%comment
%       701
%ND       9
%RA       3
%RG       2
%ZsA      2
dtype: int64